# 0.환경설정

In [ ]:
# %git clone https://github.com/Belval/TextRecognitionDataGenerator.git

#git clone https://github.com/DaveLogs/TRDG2DTRB.git

In [ ]:
import os
import subprocess
os.chdir("C:/workspace_project/AIproject/TextRecognitionDataGenerator")
os.getcwd()

In [ ]:
!create_gt_file.bat basic

In [ ]:
%pip install -r requirements.txt

In [ ]:
%pip install -r requirements-hw.txt

In [ ]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
import torch
print(torch.cuda.is_available())

# 1.학습데이터 생성

In [ ]:
import os
import subprocess
os.chdir("C:/workspace_project/AIproject/TextRecognitionDataGenerator/trdg")

In [ ]:
# TextRecognitionDataGenerator 프로젝트를 이용해 한글 학습데이터 생성하기
# TextRecognitionDataGenerator 프로젝트 root에서 실행

# training: 100,00개
!python run.py \
        --output_dir "../.././workspace/step1/trainingpr" \
        --language "ko" \
        --count 5000

In [ ]:
import os
import shutil
import random

# 이미지 파일이 있는 디렉토리
src_dir = "../.././workspace/step1/trainingpr"

# 훈련 데이터와 검증 데이터를 저장할 디렉토리
train_dir = "../.././workspace/step1/trainingprt"
valid_dir = "../.././workspace/step1/validationprt"

# 디렉토리 생성
os.makedirs(train_dir, exist_ok=True)
os.makedirs(valid_dir, exist_ok=True)

# 이미지 파일 목록 가져오기
files = os.listdir(src_dir)

# 파일 목록을 무작위로 섞기
random.shuffle(files)

# 80-20 비율로 나누기
split_idx = int(len(files) * 0.8)
train_files = files[:split_idx]
valid_files = files[split_idx:]

# 파일을 훈련 데이터와 검증 데이터 디렉토리로 이동
for file in train_files:
    shutil.move(os.path.join(src_dir, file), train_dir)

for file in valid_files:
    shutil.move(os.path.join(src_dir, file), valid_dir)

# 2.학습데이터 변환

In [ ]:
os.chdir('C:/workspace_project/AIproject/TRDG2DTRB')
os.getcwd()

In [ ]:
# TRDG2DTRB 프로젝트를 이용해 한글 학습데이터 변환하기
# TRDG2DTRB 프로젝트 root에서 실행

# train 학습데이터 변환
!python convert.py \
        --input_path ".././workspace/step1/trainingprt" \
        --output_path ".././workspace/step2/trainingprt"

# validation 학습데이터 변환
!python convert.py \
        --input_path ".././workspace/step1/validationprt" \
        --output_path ".././workspace/step2/validationprt"

# # test 학습데이터 변환
# !python3 convert.py \
#         --input_path ".././workspace/step1/testprt" \
#         --output_path ".././workspace/step2/testprt"

In [ ]:
os.chdir('C:/workspace_project/AIproject/deep-text-recognition-benchmark')
os.getcwd()

In [ ]:
%pip install patch-ng

In [ ]:
%pip install lmdb

In [ ]:
%pip install fire 

In [ ]:
# deep-text-recognition-benchmark 프로젝트로 학습데이터를 lmdb포맷으로 변환
# deep-text-recognition-benchmark 프로젝트 root에서 실행

# training 데이터 변환
!python create_lmdb_dataset.py \
        --inputPath ".././workspace/step2/trainingprt/" \
        --gtFile ".././workspace/step2/trainingprt/gt.txt" \
        --outputPath ".././workspace/step3/trainingprt"

# validation 데이터 변환
!python create_lmdb_dataset.py \
        --inputPath ".././workspace/step2/validationprt/" \
        --gtFile ".././workspace/step2/validationprt/gt.txt" \
        --outputPath ".././workspace/step3/validationprt"

# # test 데이터 변환
# !python create_lmdb_dataset.py \
#         --inputPath ".././workspace/step2/testprt/" \
#         --gtFile ".././workspace/step2/testprt/gt.txt" \
#         --outputPath ".././workspace/step3/testprt"

# 3.커스텀모델 학습

In [ ]:
%pip install natsort nltk

In [ ]:
import os
os.chdir('C:/workspace_project/AIproject/deep-text-recognition-benchmark')
os.getcwd()

In [19]:
# deep-text-recognition-benchmark 프로젝트를 이용한 모델 학습
# deep-text-recognition-benchmark 프로젝트 root에서 실행
!python train.py \
        --train_data ".././workspace/step3/trainingprt" \
        --valid_data ".././workspace/step3/validationprt" \
        --select_data / \
        --batch_ratio 1 \
        --batch_size 64 \
        --Transformation None  \
        --FeatureExtraction "VGG" \
        --SequenceModeling "BiLSTM" \
        --Prediction  "CTC" \
        --workers 0 \
        --imgH 64 \
        --imgW 200 \
        --input_channel 1 \
        --output_channel 256 \
        --hidden_size 256 \
        --saved_model ".././workspace/pre_trained_model/korean_g2.pth" \
        --FT \
        --num_iter 6000 \
        --valInterval 100

--------------------------------------------------------------------------------
dataset_root: .././workspace/step3/trainingprt
opt.select_data: ['/']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    .././workspace/step3/trainingprt	 dataset: /
sub-directory:	/.	 num samples: 309
num total samples of /: 309 x 1.0 (total_data_usage_ratio) = 309
num samples of / per batch: 64 x 1.0 (batch_ratio) = 64
--------------------------------------------------------------------------------
Total_batch_size: 64 = 64
--------------------------------------------------------------------------------
dataset_root:    .././workspace/step3/validationprt	 dataset: /
sub-directory:	/.	 num samples: 77
--------------------------------------------------------------------------------
No Transformation module specified
model input parameters 64 200 20 1 256 256 1009 25 None VGG BiLSTM CTC
loading pretrained model from .././workspace/pre_tra

In [ ]:
!python train.py \
        --train_data ".././workspace/step3/trainingprt" \
        --valid_data ".././workspace/step3/validationprt" \
        --select_data / \
        --batch_ratio 1 \
        --batch_size 64 \
	--Transformation None  \
        --FeatureExtraction "VGG" \
        --SequenceModeling "BiLSTM" \
        --Prediction  "CTC" \
        --data_filtering_off \
        --workers 0 \
        --imgH 64 \
        --imgW 200 \
        --num_iter 10000 \
        --valInterval 100

3.4 모델 테스트

In [ ]:
os.chdir('C:/workspace_project/AIproject/deep-text-recognition-benchmark')
os.getcwd()

In [ ]:
!python demo.py \
        --Transformation  None \
        --FeatureExtraction VGG \
        --SequenceModeling BiLSTM \
        --Prediction CTC \
        --image_folder demo_image/ \
        --saved_model ./saved_models/None-VGG-BiLSTM-CTC-Seed1111/best_accuracy.pth

# 4.모델 사용하기

In [ ]:
! git clone https://github.com/JaidedAI/EasyOCR.git


In [ ]:
import os
os.chdir('C:/workspace_project/AIproject/EasyOCR')

In [ ]:
!python run.py

In [ ]:
print(f"Torch CUDA available? : {torch.cuda.is_available()}")

In [ ]:
import torch
import yaml
import cv2

import easyocr
import numpy as np
import cv2
import matplotlib.pyplot as plt

from PIL import ImageFont, ImageDraw, Image

reader = easyocr.Reader(["ko", "en"], gpu=True)
result = reader.readtext(
    "C:/workspace_project/AIproject/workspace/demo_images/img_preprocessed.jpg",
    # paragraph=True
)
result2 = reader.readtext(
    "C:/workspace_project/AIproject/workspace/demo_images/img_preprocessed.jpg",
    detail=0,paragraph=True
)

img = cv2.imread(
    "C:/workspace_project/AIproject/workspace/demo_images/img_preprocessed.jpg"
)

img = Image.fromarray(img)

font = ImageFont.truetype('C:/Windows/Fonts/D2Coding-Ver1.3.2-20180524-all.ttc', 48)

draw = ImageDraw.Draw(img)

for i in result:
    x = i[0][0][0]
    y = i[0][0][1]
    w = i[0][1][0] - i[0][0][0]
    h = i[0][2][1] - i[0][1][1]

    draw.rectangle(((x, y), (x + w, y + h)), outline="red", width=2)
    draw.text((int((x + x + w) / 2), y - 40), str(i[1]), font=font, fill="red")

plt.imshow(img)
plt.show() 
# 결과를 줄바꿈으로 구분하여 출력
print(result2)

In [ ]:
%pip install yaml

In [ ]:
os.chdir('C:/workspace_project/AIproject/ocr-preprocessor-main')

In [ ]:
!python main.py

In [ ]:
import os
os.chdir('C:/workspace_project/AIproject/EasyOCR')

In [ ]:
!python run.py